In [14]:
#Importing pandas library in order to manage the datasets
import pandas as pd

In [15]:
#Reading the three datasets which contain the information regarding the deliveries made
df_ord = pd.read_csv('fact_order_lines.csv')
df_cus = pd.read_csv('dim_customers.csv')
df_pro = pd.read_csv('dim_products.csv')

In [16]:
#Converting the date columns into date format and renaming them
df_ord['scheduled_date'] = pd.to_datetime(df_ord['agreed_delivery_date'])
df_ord = df_ord.drop('agreed_delivery_date', axis=1)
df_ord['order_creation_date'] = pd.to_datetime(df_ord['order_placement_date'])
df_ord = df_ord.drop('order_placement_date', axis=1)
df_ord['delivered_date'] = pd.to_datetime(df_ord['actual_delivery_date'])
df_ord = df_ord.drop('actual_delivery_date', axis=1)

#For now we do not want to worl with the column dropped below. 
#If necessary, we can reach these results by ourselves
df_ord = df_ord.drop('In Full', axis=1)
df_ord = df_ord.drop('On Time', axis=1)
df_ord = df_ord.drop('On Time In Full', axis=1)

#Creating the column that tells the number of days between the schedueled delivery date and the order creation date 
df_ord['DB_sch_ord'] = (df_ord['scheduled_date'] - df_ord['order_creation_date']).dt.days

#Creating the column that tells the number of days between the delivery date and the scheduled delivery date 
df_ord['DB_del_sch'] = (df_ord['delivered_date'] - df_ord['scheduled_date']).dt.days

#Creating the column that tells the difference between the order quantity and the quantity that was actually delivered
df_ord['QTY_ord_del'] = df_ord['order_qty'] - df_ord['delivery_qty']

In [17]:
#Creating the function that makes the classification between groups A, B and C according to the passed %
def abc_class(perc):
    if perc > 0 and perc <= 80:
        return 'A'
    elif perc > 80 and perc <= 95:
        return 'B'
    else:
        return 'C'

In [18]:
#Creating a dataframe of all products and their respective total quantity delivered
#Throughout the orders history period provided.
df_prod_abc = df_ord.groupby('product_id').agg({'delivery_qty':'sum'})

#Ordering the data according to the descending order of the total quantity delivered
df_prod_abc = df_prod_abc.sort_values(by=['delivery_qty'], ascending=False)

#Doing the same process but for the customers instead of the products.
df_cust_abc = df_ord.groupby('customer_id').agg({'delivery_qty':'sum'})
df_cust_abc = df_cust_abc.sort_values(by=['delivery_qty'], ascending=False)

In [6]:
df_prod_abc['delivery_qty_cumsum'] = df_prod_abc['delivery_qty'].cumsum()
df_prod_abc['delivery_qty_total'] = df_prod_abc['delivery_qty'].sum()
df_prod_abc['delivery_qty_total_run_perc'] = (df_prod_abc['delivery_qty_cumsum'] / df_prod_abc['delivery_qty_total']) * 100
df_prod_abc['abc_prod'] = df_prod_abc['delivery_qty_total_run_perc'].apply(abc_class)

In [7]:
df_cust_abc['delivery_qty_cumsum'] = df_cust_abc['delivery_qty'].cumsum()
df_cust_abc['delivery_qty_total'] = df_cust_abc['delivery_qty'].sum()
df_cust_abc['delivery_qty_total_run_perc'] = (df_cust_abc['delivery_qty_cumsum'] / df_cust_abc['delivery_qty_total']) * 100
df_cust_abc['abc_cust'] = df_cust_abc['delivery_qty_total_run_perc'].apply(abc_class)

In [8]:
#Reseting the indexes of the resulting datasets containing the ABC classifications in order to
#make the merges previously.
df_prod_abc = df_prod_abc.reset_index()
df_cust_abc = df_cust_abc.reset_index()

In [9]:
#Merging all the resulting datasets into the main one, df_ord, adding to it the info regarding
#the products and customers ABC classification as well as their registration data (name and etc...) 
df_ord = pd.merge(df_ord, df_prod_abc[['product_id', 'abc_prod']], left_on='product_id', right_on='product_id', how='left')
df_ord = pd.merge(df_ord, df_cust_abc[['customer_id', 'abc_cust']], left_on='customer_id', right_on='customer_id', how='left')

df_ord = pd.merge(df_ord, df_pro[['product_id', 'product_name', 'category']], left_on='product_id', right_on='product_id', how='left')
df_ord = pd.merge(df_ord, df_cus[['customer_id', 'customer_name', 'city']], left_on='customer_id', right_on='customer_id', how='left')

In [11]:
#Creating the functions to check if and order was in time or/and in full

def fun_in_full(qty_req, qty_deli):
    if qty_deli < qty_req:
        return 0
    else:
        return 1

def fun_on_time(date_sch, date_del):
    if date_del > date_sch:
        return 0
    else:
        return 1

df_ord['in_full'] = df_ord.apply(lambda x: fun_in_full(x.order_qty, x.delivery_qty), axis=1)
df_ord['on_time'] = df_ord.apply(lambda x: fun_on_time(x.scheduled_date, x.delivered_date), axis=1)

In [13]:
df_ord.to_csv('finaldataset.csv')
df_prod_abc.to_csv('abc_prod.csv')
df_cust_abc.to_csv('abc_cust.csv')